# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port moresby,-9.4431,147.1797,26.33,79,99,1.34,PG,1706556608
1,1,college,64.8569,-147.8028,-30.03,66,100,0.00,US,1706556608
2,2,egilsstadir,65.2653,-14.3948,4.95,70,40,8.23,IS,1706556608
3,3,nikolayevsk-on-amure,53.1406,140.7300,-19.42,100,95,1.19,RU,1706556609
4,4,tabou,4.4230,-7.3528,27.32,80,3,4.42,CI,1706556609


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
city_data_df.count()

City_ID       580
City          580
Lat           580
Lng           580
Max Temp      580
Humidity      580
Cloudiness    580
Wind Speed    580
Country       575
Date          580
dtype: int64

In [5]:
clean_city_data_df = city_data_df.dropna(how='any')
clean_city_data_df.count()

City_ID       575
City          575
Lat           575
Lng           575
Max Temp      575
Humidity      575
Cloudiness    575
Wind Speed    575
Country       575
Date          575
dtype: int64

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
weather_cond = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0) 
].dropna()

# Drop any rows with null values
weather_cond = weather_cond.dropna(how='any')

# Display sample data
weather_cond

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,copiapo,-27.3667,-70.3333,21.83,37,0,4.37,CL,1706556691
90,90,woodward,36.4337,-99.3904,21.57,24,0,3.60,US,1706556697
118,118,talcahuano,-36.7167,-73.1167,25.82,38,0,3.09,CL,1706556700
124,124,atar,20.5169,-13.0499,21.51,11,0,3.82,MR,1706556701
126,126,lompoc,34.6391,-120.4579,24.41,41,0,3.09,US,1706556504
149,149,antonio enes,-16.2325,39.9086,25.33,76,0,2.46,MZ,1706556704
231,231,el granada,37.5027,-122.4694,22.73,70,0,3.09,US,1706556715
367,367,sainte-marie,-20.8968,55.5496,26.07,83,0,2.06,RE,1706556731


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_cond[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
48,copiapo,CL,-27.3667,-70.3333,37,
90,woodward,US,36.4337,-99.3904,24,
118,talcahuano,CL,-36.7167,-73.1167,38,
124,atar,MR,20.5169,-13.0499,11,
126,lompoc,US,34.6391,-120.4579,41,
149,antonio enes,MZ,-16.2325,39.9086,76,
231,el granada,US,37.5027,-122.4694,70,
367,sainte-marie,RE,-20.8968,55.5496,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = { 
    "categories":"accommodation.hotel", 
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]  
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
copiapo - nearest hotel: Hotel Diego de Almeida
woodward - nearest hotel: Sands Inn
talcahuano - nearest hotel: Club Recreativo Armada de Chile
atar - nearest hotel: فندق سكليل
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
antonio enes - nearest hotel: Hotel Quirimbas
el granada - nearest hotel: Beach House
sainte-marie - nearest hotel: Hôtel Select Océan Indien


,City,Country,Lat,Lng,Humidity,Hotel Name
48,copiapo,CL,-27.3667,-70.3333,37,Hotel Diego de Almeida
90,woodward,US,36.4337,-99.3904,24,Sands Inn
118,talcahuano,CL,-36.7167,-73.1167,38,Club Recreativo Armada de Chile
124,atar,MR,20.5169,-13.0499,11,فندق سكليل
126,lompoc,US,34.6391,-120.4579,41,Embassy Suites by Hilton Lompoc Central Coast
149,antonio enes,MZ,-16.2325,39.9086,76,Hotel Quirimbas
231,el granada,US,37.5027,-122.4694,70,Beach House
367,sainte-marie,RE,-20.8968,55.5496,83,Hôtel Select Océan Indien


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols="all",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)